##### Copyright 2018 The TensorFlow Authors.

In [25]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Custom layers

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/custom_layers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

We recommend using `tf.keras` as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.


In [26]:
import tensorflow as tf

In [27]:
print(tf.config.list_physical_devices('GPU'))

[]


## Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full [Keras](https://keras.io) API in the tf.keras package, and the Keras layers are very useful when building your own models.


In [28]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

/Users/john.zhang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


The full list of pre-existing layers can be seen in [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers). It includes Dense (a fully-connected layer),
Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [29]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [30]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<Variable path=dense_3/kernel, shape=(5, 10), dtype=float32, value=[[-0.22994575 -0.49147144 -0.5178481  -0.27209103 -0.53934777  0.60627216
    0.15775079 -0.1457338  -0.0314008  -0.43561083]
  [ 0.28768986  0.2811008  -0.57415473 -0.3440265  -0.44788826 -0.11124724
   -0.3741258   0.10411507  0.44845372  0.1353634 ]
  [ 0.07231742  0.16962123  0.39895624 -0.2166959   0.2167995   0.624466
    0.46778578 -0.0529986   0.00241172 -0.5738483 ]
  [-0.19739279 -0.62059325  0.11512828 -0.18054843 -0.49070558  0.36658907
   -0.05692142  0.40571266 -0.6128929   0.08243716]
  [-0.24522734  0.09613383  0.38760465  0.41042238 -0.15914589 -0.33588615
    0.534705    0.29389572  0.37770325  0.05922937]]>,
 <Variable path=dense_3/bias, shape=(10,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]>]

In [31]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<Variable path=dense_3/kernel, shape=(5, 10), dtype=float32, value=[[-0.22994575 -0.49147144 -0.5178481  -0.27209103 -0.53934777  0.60627216
    0.15775079 -0.1457338  -0.0314008  -0.43561083]
  [ 0.28768986  0.2811008  -0.57415473 -0.3440265  -0.44788826 -0.11124724
   -0.3741258   0.10411507  0.44845372  0.1353634 ]
  [ 0.07231742  0.16962123  0.39895624 -0.2166959   0.2167995   0.624466
    0.46778578 -0.0529986   0.00241172 -0.5738483 ]
  [-0.19739279 -0.62059325  0.11512828 -0.18054843 -0.49070558  0.36658907
   -0.05692142  0.40571266 -0.6128929   0.08243716]
  [-0.24522734  0.09613383  0.38760465  0.41042238 -0.15914589 -0.33588615
    0.534705    0.29389572  0.37770325  0.05922937]]>,
 <Variable path=dense_3/bias, shape=(10,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]>)

## Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing:

1. `__init__` , where you can do all input-independent initialization
2. `build`, where you know the shapes of the input tensors and can do the rest of the initialization
3. `call`, where you do the forward computation

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. However, the advantage of creating them in `build` is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.

In [32]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    num_input = int(input_shape[-1])
    self.kernel = self.add_weight(name="kernel", shape=(num_input, self.num_outputs))

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [33]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [34]:
print([var.name for var in layer.trainable_variables])

['kernel']


In [35]:
layer.variables

[<Variable path=my_dense_layer_6/kernel, shape=(5, 10), dtype=float32, value=[[-0.30172804  0.14546871  0.24062312 -0.27499026 -0.30947182  0.16124982
   -0.43139374  0.5152355   0.31808007 -0.48748428]
  [ 0.40717894  0.06895739 -0.3324798   0.08195752 -0.26975062 -0.19407362
    0.4931348   0.2719431   0.6150362   0.3422125 ]
  [-0.2745828  -0.5344192  -0.38614386  0.07755208 -0.6310052  -0.59519327
   -0.37392482  0.6161409  -0.3784892   0.171983  ]
  [ 0.60972184  0.52250355  0.45673782 -0.08851713 -0.3969063   0.06354737
    0.08876956 -0.36090598 -0.6155195  -0.5012845 ]
  [ 0.36792117  0.14378637  0.3812334  -0.17208284 -0.545474    0.54971355
    0.01765966  0.59202915  0.48775584  0.6001201 ]]>]

Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in `tf.keras.layers`, consider filing a [github issue](http://github.com/tensorflow/tensorflow/issues/new) or, even better, sending us a pull request!

## Models: Composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from `keras.Model` when you need the model methods like: `Model.fit`,`Model.evaluate`, and `Model.save` (see [Custom Keras layers and models](https://www.tensorflow.org/guide/keras/custom_layers_and_models) for details).

One other feature provided by `keras.Model` (instead of `keras.layers.Layer`) is that in addition to tracking variables, a `keras.Model` also tracks its internal layers, making them easier to inspect.

For example here is a ResNet block:

In [ ]:

class ResnetBlock(tf.keras.Model):
    def __init__(self, kernal_size, filters):
        super(ResnetBlock, self).__init__()
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2d(filters1, (1,1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2d(filters2, kernal_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2d(filters3, (1,1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training=False):
        x = self.conv2a(inputs)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        # x = tf.nn.relu(x)             
        
        x += inputs
        x = tf.nn.relu(x)  
        
        return x 
    

In [20]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [21]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [22]:
block.layers

[<Conv2D name=conv2d, built=True>,
 <BatchNormalization name=batch_normalization, built=True>,
 <Conv2D name=conv2d_1, built=True>,
 <BatchNormalization name=batch_normalization_1, built=True>,
 <Conv2D name=conv2d_2, built=True>,
 <BatchNormalization name=batch_normalization_2, built=True>]

In [23]:
len(block.variables)

18

In [24]:
block.summary()

Model: ""

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (1, 2, 3, 1)           │             4 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (1, 2, 3, 1)           │             4 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (1, 2, 3, 2)           │             4 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (1, 2, 3, 2)           │             8 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (1, 2, 3, 3)           │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (1, 2, 3, 3)           │            12 │
│ (BatchNormalization)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41 (164.00 B)

 Trainable params: 29 (116.00 B)

 Non-trainable params: 12 (48.00 B)

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using `tf.keras.Sequential`:

In [ ]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

In [ ]:
my_seq.summary()

# Next steps

Now you can go back to the previous notebook and adapt the linear regression example to use layers and models to be better structured.